In [6]:
# Libraries for data management
import os
import json
import pandas as pd
import numpy as np
import ast

#FOR PROCESSING
import nltk
import re
nltk.download("omw-1.4")

#FORR W2V
import gensim
import gensim.downloader as gensim_api

#FOR PLOTTING
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#FOR BERT MODEL -> TO STUDY BERT MODEL
import transformers


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jesustellez/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
""" TIME SERIES WITH #VIDEOS, LIKES, COMMENTS """

class EDA_Analysis():

    def __init__(self) -> None:
        def data_read_csv(path, title):
            video = path + "/" + title
            df_ = pd.read_csv(video)

            return df_

        path = "/Users/jesustellez/Desktop/aiDynamics/API Data extraction/Data"
        df_comments = data_read_csv(path, "comments_eeuu.csv")
        self.df_comments = df_comments
        df_videos = data_read_csv(path, "videos_eeuu.csv")
        df_videos["Date"] = df_videos["publishedAt"].str.split("T", expand=True)[0]
        self.df_videos = df_videos
        df_calendar = data_read_csv(path, "calendar.csv")
        df_calendar["Week-Year"] = df_calendar["Week"].astype(str) + "-" + df_calendar["Year"].astype(str)
        self.df_calendar = df_calendar

    def __returndata__(self):
        return self.df_videos, self.df_calendar, self.df_comments
    
    def EDA_timely(self, period):

        df_ts_analysis = self.df_videos.merge(
            self.df_calendar[["Date", "Day", "Week","Month", "Month-Year", "Week-Year"]], 
            on="Date", 
            how="left")
        
        df_ts_analysis_wy = df_ts_analysis.groupby(period, as_index=False).agg({"viewCount":"sum", "video_id":"count","commentCount":"sum"})
        return df_ts_analysis_wy

    def EDA_time_graph(self, period, fields, df_analysis):

        fig = go.Figure()
        for i in fields:
            fig.add_trace(go.Scatter(x=df_analysis[period], y=df_analysis[i]))

        fig.show()

df_videos, df_calendar, df_comments = EDA_Analysis().__returndata__()


In [8]:
"""
THIS FIRST PART I AM GOING TO REDUCE THE NUMBER OF TAGS TO TRY TO CORRELATE THE VIDEOS WITH TOPICS (IN EXAMPLE, WAR, POLITICS, TECHNOLOGY, ETC)
"""
def clean_tags(text, flg_stemm = False, flg_lemm=True, lst_stopwords = None):
    ## Clean (convert to lowercase and remove punctuation and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower())

    ## Tolenize (Convert from string to List)
    lst_text = text.split()

    ## Remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    ## Stemming (to remove -ly, -ing, etc.)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (Convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## Back to string from list
    text = " ".join(lst_text)
    return text

In [9]:
## PREPROCESSING OF DATA SET
df_videos["tags"] = df_videos.tags.str.split(",")
df_videos = df_videos.explode("tags")

## CREATION OF STOPWORDS WITH NLTK LIBRARY
lst_stopwords = nltk.corpus.stopwords.words("english")


df_videos["tags_clean"] = df_videos["tags"].apply(lambda x: clean_tags(
    x,
    flg_stemm=False,
    flg_lemm=True,
    lst_stopwords=lst_stopwords
))


In [13]:
df_videos.tags_clean.unique()

array(['1917 espionage act', '1918 sedition act', 'news', ...,
       'jared polis', 'middle fork fire', 'new year fire'], dtype=object)